In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\paulb\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,reduce,fact,drive,1791,nothingness,walk,quieter,Mrs,park,unconnected,...,aggravation,seriously,objection,steady,importance,table,disposed,care,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [7]:
word_counts.shape

(444, 1614)

## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [8]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9736842105263158

Test set score: 0.8314606741573034


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1612) (266,)
Training set score: 0.9699248120300752

Test set score: 0.8764044943820225


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [10]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9661654135338346

Test set score: 0.8146067415730337


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [11]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [12]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [13]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [14]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
done


In [15]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7235772357723578


col_0,Austen,Carroll
row_0,,
Austen,158,12
Carroll,56,20


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

#### Attempt 1: Trying Support Vector Classifier

In [16]:
# Trying SVC model for classifier
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


(266, 1612) (266,)
Training set score: 0.7142857142857143

Test set score: 0.702247191011236


#### Attempt 2: Making More Features
##### A. Adding Punctuation

In [17]:
# Redefining function above
def bag_of_words(text):
    allwords = [token.lemma_
                for token in text
                if not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [18]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [19]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.shape

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


(444, 1627)

In [20]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [21]:
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1625) (266,)
Training set score: 0.9699248120300752

Test set score: 0.8764044943820225


Adding punctuation caused no change to the accuracy.
##### B. Adding Sentence Length

In [22]:
X = word_counts.drop(['text_sentence','text_source'], 1)
X['sents_len'] = X.sum(1)

Y = word_counts['text_source']
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [23]:
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1626) (266,)
Training set score: 0.9736842105263158

Test set score: 0.8707865168539326


Adding sentence length caused a decrease in accuracy.
##### C. Adding Punctuation Count

In [27]:
# Counting number of punctuation
import string
count = lambda l1,l2: sum([1 for x in l1 if x in l2])
counter = []

for sentence in word_counts.text_sentence:
    counter.append(count(str(sentence),set(string.punctuation)))

X = pd.DataFrame(X)
X['punc_count'] = counter

In [28]:
Y = word_counts['text_source']
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [29]:
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1627) (266,)
Training set score: 0.9736842105263158

Test set score: 0.8707865168539326


Once again, no change to the score.
##### D. Adding Contextual Features

In [30]:
# Finding average of surrounding sentences for contextual feature
contextual_length = []
for index in word_counts.text_sentence.index:
    if index == 0:
        current = word_counts.text_sentence[index]
        latter = word_counts.text_sentence[index+1]
        sent_mean = (len(current) + len(latter))/2
        contextual_length.append(sent_mean)
    elif index == 443:
        current = word_counts.text_sentence[index]
        former = word_counts.text_sentence[index-1]
        sent_mean = (len(current) + len(former))/2
        contextual_length.append(sent_mean)
    else:
        former = word_counts.text_sentence[index-1]
        latter = word_counts.text_sentence[index+1]
        sent_mean = (len(former) + len(latter))/2
        contextual_length.append(sent_mean)

In [33]:
X = word_counts.drop(['text_sentence','text_source'], 1)
X['context_length'] = contextual_length

Y = word_counts['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [34]:
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1626) (266,)
Training set score: 0.9699248120300752

Test set score: 0.8764044943820225


In [36]:
word_counts['context_length'] = contextual_length
print(word_counts[word_counts.text_source == 'Carroll'].context_length.mean())
print(word_counts[word_counts.text_source == 'Austen'].context_length.mean())

26.37984496124031
30.39047619047619


As is noticeable above, there is no considerable difference in sentence length between the two works. For this reason, there is no change to the accuracy of the model.
##### E. Adding Parts-of-Speech Features

In [37]:
from collections import Counter

# Loading all available parts of speech in a sentence
full_pos = pd.DataFrame(columns=['ADJ','ADP','ADV','AUX','CCONJ','CONJ','DET',
                                     'INTJ','NOUN','NUMV','PART','PRON','PROPN',
                                     'PUNCT','SCONJ','SPACE','SYM','VERB','X'])

# Loop to add all parts of speech within the sentences to a DataFrame
for sentences in word_counts['text_sentence']:
    # Extracting POS from words in sentences
    parts_of_speech = [word.pos_ for word in sentences]
    # Counting amount of times certain POS is used in a sentence
    counted = pd.Series(Counter(parts_of_speech))
    # Loading empty POS Series for filling
    pos_list = pd.Series(index=['ADJ','ADP','ADV','AUX','CCONJ','CONJ','DET',
                                'INTJ','NOUN','NUMV','PART','PRON','PROPN',
                                'PUNCT','SCONJ','SPACE','SYM','VERB','X'])
    # Appending counted POS values to empty Series
    pos_list = pos_list.append(counted)
    # Grouping matching indexes for easy loading and filling NaN values
    grouped = pos_list.groupby(level=0)
    all_pos = grouped.last()
    all_pos = all_pos.fillna(0)
    # Appending all counted POS values of sentences to entire DataFrame
    full_pos = full_pos.append(all_pos,1)

full_pos = full_pos.fillna(0)
full_pos.head()

,ADJ,ADP,ADV,AUX,CCONJ,CONJ,DET,INTJ,NOUN,NUMV,PART,PRON,PROPN,PUNCT,SCONJ,SPACE,SYM,VERB,X,NUM
0,1.0,8.0,3.0,0.0,6.0,0.0,7.0,0.0,11.0,0.0,2.0,4.0,2.0,10.0,0.0,0.0,0.0,13.0,0.0,0.0
1,6.0,8.0,7.0,0.0,2.0,0.0,7.0,0.0,8.0,0.0,1.0,4.0,2.0,7.0,0.0,0.0,0.0,11.0,0.0,0.0
2,1.0,4.0,6.0,0.0,1.0,0.0,3.0,0.0,2.0,0.0,1.0,2.0,2.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
all_counts = pd.concat([word_counts, full_pos], axis=1)

In [39]:
all_counts.head()

,reduce,fact,drive,1791,nothingness,walk,quieter,",",Mrs,park,...,PART,PRON,PROPN,PUNCT,SCONJ,SPACE,SYM,VERB,X,NUM
0,0,0,0,0,0,0,0,0,0,0,...,2.0,4.0,2.0,10.0,0.0,0.0,0.0,13.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,1.0,4.0,2.0,7.0,0.0,0.0,0.0,11.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,1.0,2.0,2.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
all_counts['contextual_length'] = contextual_length
X = all_counts.drop(['text_sentence','text_source'], 1)

Y = word_counts['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [41]:
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1647) (266,)
Training set score: 0.981203007518797

Test set score: 0.8707865168539326


In [42]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X, Y, cv=5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

array([0.86516854, 0.84269663, 0.91011236, 0.85393258, 0.78409091])

In [43]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

k_scores = []

for k in range(125,175,1):
    X_new = SelectKBest(chi2, k=k).fit_transform(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.4, random_state=0)
    lr = LogisticRegression(penalty='l2', solver='liblinear')
    train = lr.fit(X_train, y_train)
    k_scores.append(lr.score(X_test, y_test))

In [44]:
test = pd.DataFrame(k_scores)
test[test == test.max()].dropna()

,0
0,0.88764
1,0.88764
2,0.88764
3,0.88764
4,0.88764
5,0.88764
6,0.88764
10,0.88764
12,0.88764
13,0.88764


In [47]:
X_new = SelectKBest(chi2, k=125).fit_transform(X, Y)
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.4, random_state=0)
lr = LogisticRegression(penalty='l2')
train = lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8876404494382022

While the model didn't perform above the 90% target, we were able to improve the model's score by adding some features and doing feature reduction to prevent overfitting.

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [83]:
# Clean the Milton data.
milton = gutenberg.raw('milton-paradise.txt')
milton = re.sub(r'VOLUME \w+', '', milton)
milton = re.sub(r'CHAPTER \w+', '', milton)
milton = text_cleaner(milton[:int(len(milton)/12)])
print(milton[:100])

Book I Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought 


In [84]:
# Parse our cleaned data.
milton_doc = nlp(milton)

In [85]:
# Group into sentences.
paradise_sents = [[sent, "Milton"] for sent in milton_doc.sents]

In [86]:
paradise_sentences = pd.DataFrame(paradise_sents)
paradise_bow = bow_features(paradise_sentences, common_words)

print(paradise_sentences.shape)
print(paradise_bow.shape)

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
(276, 2)
(276, 1627)
